# Modelos Seq2Seq

Modelos Seq2Seq mapeiam sequências de tokens para outras sequências de tokens. Em geral, eles têm um par encoder-decoder. O encoder mapeia uma sequência inteira para 

In [9]:
from keras.layers import Input, Dense, Activation, TimeDistributed, Softmax, TextVectorization, Reshape, RepeatVector, Conv1D, Bidirectional, AveragePooling1D, UpSampling1D, Embedding, Concatenate, GlobalAveragePooling1D, LSTM, Multiply
from keras.models import Model
import tensorflow as tf

In [10]:
# Refazer estrutura do dataset: uma frase por arquivo, arquivos em um diretorio
import pandas as pd
import os
import re
from tqdm import tqdm

#df = pd.read_csv('datasets/IMDB Dataset.csv')
#DATASET_DIR = './frases_imdb/'
DATASET_DIR = './datasets/frases/'
#DATASET_DIR = './datasets/frases_classificacao/'


In [11]:

# n_texto = 0
# for i in tqdm(range(len(df))):
#     texto = df.iloc[i]['review']
#     #print(texto)
#     texto = texto.replace('<br />', '.')
#     #texto = re.sub(r'[\n \t][\n \t][\n \t]+', '.', texto)
#     texto = re.split(r'[.!?\n\t-]', texto)
#     #print(texto)
#     for t in texto:

#         t = t.lstrip().rstrip()
#         if len(t) > 5:
#             #print(t)
#             fname = 'review_' + str(n_texto) + '.txt'
#             with open(DATASET_DIR + fname, 'w') as f:
#                 f.write(t)
#             n_texto += 1

# print("Criei textos:", n_texto)


In [12]:
# Ler um dataset e fazer batches
from tensorflow.keras.utils import text_dataset_from_directory

dataset = text_dataset_from_directory(
    DATASET_DIR,
    labels=None,
    label_mode='int',
    class_names=None,
    batch_size=512,
    max_length=None,
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    follow_links=False
)


Found 22516 files belonging to 1 classes.


In [35]:

@tf.function
def adicionar_inicio_e_fim_de_sentenca(x):
    sos = tf.convert_to_tensor( 'SOS', dtype=tf.string)
    x = tf.strings.join( (sos, x))
    return x

In [36]:
sos = tf.convert_to_tensor( 'SOS', dtype=tf.string)
print(sos, sos.shape)


tf.Tensor(b'SOS', shape=(), dtype=string) ()


In [37]:
from keras.layers import Input, TextVectorization
from keras.models import Model
vocab_size = 10000
vectorize_layer = TextVectorization(max_tokens=vocab_size, output_sequence_length=15)
vectorize_layer.adapt(dataset.map(adicionar_inicio_e_fim_de_sentenca))

(None,)
(None,)


In [ ]:
vectorize_layer.get_vocabulary()[0:10]

['', '[UNK]', 'the', 'and', 'of', 'a', 'to', 'in', 'is', 'that']

In [200]:
from tensorflow import one_hot


def meu_preprocessador(x):
    vector = vectorize_layer(x)
    return vector, vector



In [158]:
import tensorflow as tf
import numpy as np
vector = tf.convert_to_tensor( [[1, 2, 3, 0], [3, 2, 1, 0]] )
one_hot(vector, 5)

<tf.Tensor: shape=(2, 4, 5), dtype=float32, numpy=
array([[[0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.]],

       [[0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.]]], dtype=float32)>

In [344]:
def encoder(seq_len, latent_dim, vocab_size):
    input_layer = Input(shape=(seq_len,))
    x = input_layer
    x = Embedding(vocab_size, latent_dim, name='embedding')(x)
    x = LSTM(latent_dim, kernel_initializer='glorot_uniform', activation='relu', go_backwards=True)(x)
    return Model(input_layer, x)

def decoder(seq_len, latent_dim, vocab_size):
    from tensorflow import zeros_like
    input_x = Input(shape=(latent_dim))
    x = RepeatVector(seq_len)(input_x)
    x = LSTM(latent_dim, return_sequences=True)(x)
    x = TimeDistributed(Dense(vocab_size, kernel_initializer='glorot_uniform', activation='relu'))(x)
    Z = zeros_like(x)[:,:,0:2]
    x = Concatenate()( (Z, x[:,:,2:]) ) 
    x = Activation('softmax')(x)
    return Model(input_x, x)

def autoencoder(encoder, decoder):
    in_shape = encoder.layers[0].input_shape[0][1:]
    #print(in_shape)
    input_layer = Input(shape=in_shape)
    x = input_layer
    x = encoder(x)
    x = decoder(x)
    return Model(input_layer, x)

import keras
enc = encoder(15, 20, vocab_size)
enc.summary()
dec = decoder(15, 20, vocab_size)
dec.summary()
ae = autoencoder(enc, dec)
opt = keras.optimizers.SGD(learning_rate=1, momentum=0.9)
ae.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
ae.summary()


Model: "model_221"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_381 (InputLayer)      [(None, 15)]              0         
                                                                 
 embedding (Embedding)       (None, 15, 20)            200000    
                                                                 
 lstm_266 (LSTM)             (None, 20)                3280      
                                                                 
Total params: 203,280
Trainable params: 203,280
Non-trainable params: 0
_________________________________________________________________
Model: "model_222"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_382 (InputLayer)         [(None, 20)]         0           []                               

In [348]:
textos = ["This is a test", "This is another test","Movies are so bad", "There are thousands of ways to write"]
X_ = vectorize_layer(textos)
from tensorflow.data import AUTOTUNE
history = ae.fit(X_ ,X_ ,epochs=1000, verbose=1)

Epoch 1/1000
1/1 [==============================] - 0s 60ms/step - loss: 6.6018 - accuracy: 0.8167
Epoch 2/1000
1/1 [==============================] - 0s 57ms/step - loss: 6.6018 - accuracy: 0.8667
Epoch 3/1000
1/1 [==============================] - 0s 52ms/step - loss: 6.6018 - accuracy: 0.8500
Epoch 4/1000
1/1 [==============================] - 0s 60ms/step - loss: 6.6018 - accuracy: 0.8500
Epoch 5/1000
1/1 [==============================] - 0s 61ms/step - loss: 6.6018 - accuracy: 0.8500
Epoch 6/1000
1/1 [==============================] - 0s 60ms/step - loss: 6.6018 - accuracy: 0.8333
Epoch 7/1000
1/1 [==============================] - 0s 55ms/step - loss: 6.6018 - accuracy: 0.8333
Epoch 8/1000
1/1 [==============================] - 0s 70ms/step - loss: 6.6018 - accuracy: 0.8500
Epoch 9/1000
1/1 [==============================] - 0s 64ms/step - loss: 6.6018 - accuracy: 0.8500
Epoch 10/1000
1/1 [==============================] - 0s 62ms/step - loss: 6.6018 - accuracy: 0.8500
Epoch 11/

In [349]:
textos = ["This is a test", "This is another test","Movies are so bad", "There are thousands of ways to write", "Never seen this sentence", "Crazy stuff going on in this one yay"]
X_ = vectorize_layer(textos)
print(X_)
#X_ = one_hot(X_, vocab_size)
X_transf = ae.predict(X_)
X_transf = X_transf.argmax(axis=-1)
print(X_transf)
X_texts = []
print(len(vectorize_layer.get_vocabulary()))
for i in range(X_transf.shape[0]):
    this_text = [vectorize_layer.get_vocabulary()[j] for j in X_transf[i,:]]
    X_texts.append(this_text)
    print(this_text)

tf.Tensor(
[[  14    8    5 1841    0    0    0    0    0    0    0    0    0    0
     0]
 [  14    8  154 1841    0    0    0    0    0    0    0    0    0    0
     0]
 [ 105   25   41  111    0    0    0    0    0    0    0    0    0    0
     0]
 [  54   25 1970    4  721    6 1053    0    0    0    0    0    0    0
     0]
 [ 119  126   14 3653    0    0    0    0    0    0    0    0    0    0
     0]
 [1114  639  159   19    7   14   33    1    0    0    0    0    0    0
     0]], shape=(6, 15), dtype=int64)
1/1 [==============================] - 0s 53ms/step
[[  14    8    5 1841 1053    0    0    0    0    0    0    0    0    0
     0]
 [  14    8  154 1841 1841    0    0    0    0    0    0    0    0    0
     0]
 [ 105   41   41  111  111    0    0    0    0    0    0    0    0    0
     0]
 [  54 1970 1970    4  721    6 1053 1053    0    0    0    0    0    0
     0]
 [  54  154  154    4  721 1053 1053 9662    0    0    0    0 3781 3781
  3781]
 [  54  154   41    4  721 

In [350]:
from tensorflow.data import AUTOTUNE
history = ae.fit(dataset.map(meu_preprocessador, num_parallel_calls=AUTOTUNE),epochs=5, verbose=1)

Epoch 1/5
44/44 [==============================] - 122s 3s/step - loss: 8.3839 - accuracy: 0.0525
Epoch 2/5
44/44 [==============================] - 127s 3s/step - loss: 7.9096 - accuracy: 0.0650
Epoch 3/5
44/44 [==============================] - 120s 3s/step - loss: 7.8728 - accuracy: 0.0647
Epoch 4/5
44/44 [==============================] - 117s 3s/step - loss: 7.7622 - accuracy: 0.0705
Epoch 5/5
44/44 [==============================] - 116s 3s/step - loss: 7.7468 - accuracy: 0.0715


In [343]:
textos = ["This is a test", "This is another test", "Nature works towards my movies"]
X_ = vectorize_layer(textos)
#X_ = one_hot(X_, vocab_size)
X_transf = ae.predict(X_)
#X_transf[:,:,0:2]*=0.0
print(X_transf[0,:,:])
X_transf = X_transf.argmax(axis=-1)
print(X_transf)
X_texts = []
print(len(vectorize_layer.get_vocabulary()))
for i in range(X_transf.shape[0]):
    this_text = [vectorize_layer.get_vocabulary()[j] for j in X_transf[i,:]]
    X_texts.append(this_text)
    print(this_text)

1/1 [==============================] - 0s 129ms/step
[[5.9316018e-05 5.9316018e-05 5.1455390e-02 ... 5.9316018e-05
  5.9316018e-05 5.9316018e-05]
 [6.1756502e-05 6.1756502e-05 5.8943361e-02 ... 6.1756502e-05
  6.1756502e-05 6.1756502e-05]
 [6.1183382e-05 6.1183382e-05 5.9257813e-02 ... 6.1183382e-05
  6.1183382e-05 6.1183382e-05]
 ...
 [6.1075843e-05 6.1075843e-05 5.9291888e-02 ... 6.1075843e-05
  6.1075843e-05 6.1075843e-05]
 [6.1075843e-05 6.1075843e-05 5.9291888e-02 ... 6.1075843e-05
  6.1075843e-05 6.1075843e-05]
 [6.1075843e-05 6.1075843e-05 5.9291888e-02 ... 6.1075843e-05
  6.1075843e-05 6.1075843e-05]]
[[10  2  2  2  2  2  2  2  2  2  2  2  2  2  2]
 [10  2  2  2  2  2  2  2  2  2  2  2  2  2  2]
 [10  2  2  2  2  2  2  2  2  2  2  2  2  2  2]]
10000
['br', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the']
['br', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the']
['br', 'the', 'the', '

In [175]:
emb = enc.predict(X_)
print(emb)

1/1 [==============================] - 1s 773ms/step
[array([[7.8085566e+00, 5.6553304e-01, 3.8190581e-05, 2.2037330e+00,
        4.7765470e+00, 7.5060329e+00, 2.0977554e+00, 2.9130390e+00,
        8.5471945e+00, 2.0062025e+00, 5.1681623e+00, 2.8012955e+00,
        3.2420509e+00, 4.3727036e+00, 1.9953672e+00, 1.9011010e+00,
        3.3039753e+00, 1.5840873e-04, 2.6989748e+00, 6.5706081e+00],
       [7.1461568e+00, 5.3229219e-01, 2.6556045e-05, 2.0613029e+00,
        4.3520136e+00, 7.0070066e+00, 2.0220585e+00, 2.6708205e+00,
        7.8382716e+00, 1.8460445e+00, 4.9928484e+00, 2.7395306e+00,
        3.1051791e+00, 4.2326617e+00, 1.9570433e+00, 1.8441373e+00,
        3.1642699e+00, 1.5548670e-04, 2.6346014e+00, 6.3296213e+00],
       [7.2349534e+00, 5.3994459e-01, 2.9533168e-05, 2.0841260e+00,
        4.4055982e+00, 7.0716581e+00, 2.0339315e+00, 2.6930943e+00,
        7.9194403e+00, 1.8637562e+00, 5.4255009e+00, 2.9253490e+00,
        3.2184579e+00, 4.6237664e+00, 2.0762329e+00, 1.93800

OrderedDict([('review', <tf.Tensor: shape=(16,), dtype=string, numpy=
array([b'The first point that calls the attention in "For Ever Mozart" is the absence of a plot summary in IMDb. The explanation is simple since there is no story, screenplay, plot or whatever might recall the minimum structure of a movie. Jean-Luc Godard is one of the most overrated and pretentious directors of the cinema industry and this pointless crap is among his most hermetic films. I believe that neither himself has understood what is this story about; but there are intellectuals that elucubrate to justify or explain this messy movie, and it is funny to read their reviews. <br /><br />My vote is one.<br /><br />Title (Brazil): "Para Sempre Mozart" ("Forever Mozart")',
       b'Where do you begin with a movie as bad as this?<br /><br />Do you mention the cast of unlikeable heroes? The over-the-top acting? The dreadful script?<br /><br />No. You just say that anyone who pays money to see a film as poor as this n